In [33]:
import re
import json
import stanza

import numpy as np
import pandas as pd

from itertools import chain
from ast import literal_eval

from preprocessing_functions import load_txt_as_lst, split_txt, \
run_stanza, flatten_list, segment_series

ModuleNotFoundError: No module named 'preprocessing_functions'

In [3]:
top10_path = "/home/craig.car/repos/chiron/align_texts_project/data/top10-redo.jsonl"
top10_df = pd.read_json(top10_path, lines=True)

In [5]:
top10_df.head()

,id,book,seq,loc,text
0,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.1,urn:cts:latinLit:phi0550.phi001.perseus-lat1,0,urn:cts:latinLit:phi0550.phi001:1.1,"Aeneadum genetrix, hominum divomque voluptas,\n"
1,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.2,urn:cts:latinLit:phi0550.phi001.perseus-lat1,1,urn:cts:latinLit:phi0550.phi001:1.2,"alma Venus, caeli subter labentia signa\n"
2,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.3,urn:cts:latinLit:phi0550.phi001.perseus-lat1,2,urn:cts:latinLit:phi0550.phi001:1.3,"quae mare navigerum, quae terras frugiferentis\n"
3,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.4,urn:cts:latinLit:phi0550.phi001.perseus-lat1,3,urn:cts:latinLit:phi0550.phi001:1.4,"concelebras, per te quoniam genus omne animant..."
4,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.5,urn:cts:latinLit:phi0550.phi001.perseus-lat1,4,urn:cts:latinLit:phi0550.phi001:1.5,concipitur visitque exortum lumina solis:\n


In [6]:
top10_df.groupby("book").count()

,id,seq,loc,text
book,,,,
urn:cts:greekLit:tlg0011.tlg002.perseus-grc2,1257,1257,1257,1257
urn:cts:greekLit:tlg0012.tlg001.perseus-grc2,15686,15686,15686,15686
urn:cts:greekLit:tlg0012.tlg002.perseus-grc2,12107,12107,12107,12107
urn:cts:greekLit:tlg0085.tlg005.opp-grc3,1643,1643,1643,1643
urn:cts:greekLit:tlg0085.tlg005.perseus-grc2,1649,1649,1649,1649
urn:cts:latinLit:phi0550.phi001.perseus-lat1,7420,7420,7420,7420
urn:cts:latinLit:phi0959.phi006.perseus-lat2,11927,11927,11927,11927


In [7]:
lucretius_lat = top10_df.loc[top10_df["book"]=="urn:cts:latinLit:phi0550.phi001.perseus-lat1"]

In [10]:
lucretius_lat

,id,book,seq,loc,text
0,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.1,urn:cts:latinLit:phi0550.phi001.perseus-lat1,0,urn:cts:latinLit:phi0550.phi001:1.1,"Aeneadum genetrix, hominum divomque voluptas,\n"
1,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.2,urn:cts:latinLit:phi0550.phi001.perseus-lat1,1,urn:cts:latinLit:phi0550.phi001:1.2,"alma Venus, caeli subter labentia signa\n"
2,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.3,urn:cts:latinLit:phi0550.phi001.perseus-lat1,2,urn:cts:latinLit:phi0550.phi001:1.3,"quae mare navigerum, quae terras frugiferentis\n"
3,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.4,urn:cts:latinLit:phi0550.phi001.perseus-lat1,3,urn:cts:latinLit:phi0550.phi001:1.4,"concelebras, per te quoniam genus omne animant..."
4,urn:cts:latinLit:phi0550.phi001.perseus-lat1:1.5,urn:cts:latinLit:phi0550.phi001.perseus-lat1,4,urn:cts:latinLit:phi0550.phi001:1.5,concipitur visitque exortum lumina solis:\n
...,...,...,...,...,...
7415,urn:cts:latinLit:phi0550.phi001.perseus-lat1:6...,urn:cts:latinLit:phi0550.phi001.perseus-lat1,7415,urn:cts:latinLit:phi0550.phi001:6.1282,inque aliis alium populum sepelire suorum\n
7416,urn:cts:latinLit:phi0550.phi001.perseus-lat1:6...,urn:cts:latinLit:phi0550.phi001.perseus-lat1,7416,urn:cts:latinLit:phi0550.phi001:6.1283,certantes; lacrimis lassi luctuque redibant;\n
7417,urn:cts:latinLit:phi0550.phi001.perseus-lat1:6...,urn:cts:latinLit:phi0550.phi001.perseus-lat1,7417,urn:cts:latinLit:phi0550.phi001:6.1284,inde bonam partem in lectum maerore dabantur;\n
7418,urn:cts:latinLit:phi0550.phi001.perseus-lat1:6...,urn:cts:latinLit:phi0550.phi001.perseus-lat1,7418,urn:cts:latinLit:phi0550.phi001:6.1285,"nec poterat quisquam reperiri, quem neque morb..."


Each row corresponds to a line of text in [Perseus version](http://www.perseus.tufts.edu/hopper/text?doc=Perseus%3Atext%3A1999.02.0130%3Abook%3D1%3Acard%3D1).

# Get Lucretius sentences, no section markings

In [19]:
# remove ending \n
lucretius_lat_lst = list(lucretius_lat["text"].str.strip("\n"))

In [20]:
lucretius_lat_lst[:10]

['Aeneadum genetrix, hominum divomque voluptas,',
 'alma Venus, caeli subter labentia signa',
 'quae mare navigerum, quae terras frugiferentis',
 'concelebras, per te quoniam genus omne animantum',
 'concipitur visitque exortum lumina solis:',
 'te, dea, te fugiunt venti, te nubila caeli',
 'adventumque tuum, tibi suavis daedala tellus',
 'summittit flores, tibi rident aequora ponti',
 'placatumque nitet diffuso lumine caelum.',
 'nam simul ac species patefactast verna diei']

In [21]:
lucretius_lat_str = " ".join(lucretius_lat_lst)

In [23]:
lucretius_lat_str[:1000]

'Aeneadum genetrix, hominum divomque voluptas, alma Venus, caeli subter labentia signa quae mare navigerum, quae terras frugiferentis concelebras, per te quoniam genus omne animantum concipitur visitque exortum lumina solis: te, dea, te fugiunt venti, te nubila caeli adventumque tuum, tibi suavis daedala tellus summittit flores, tibi rident aequora ponti placatumque nitet diffuso lumine caelum. nam simul ac species patefactast verna diei et reserata viget genitabilis aura favoni, aeriae primum volucris te, diva, tuumque significant initum perculsae corda tua vi. inde ferae pecudes persultant pabula laeta et rapidos tranant amnis: ita capta lepore te sequitur cupide quo quamque inducere pergis. denique per maria ac montis fluviosque rapacis frondiferasque domos avium camposque virentis omnibus incutiens blandum per pectora amorem efficis ut cupide generatim saecla propagent. quae quoniam rerum naturam sola gubernas nec sine te quicquam dias in luminis oras exoritur neque fit laetum nequ

In [26]:
# load stanza model for Latin
lang_ = "la"
stanza_model_ = stanza.Pipeline(lang=lang_, processors='tokenize', use_gpu=True)

2023-06-15 11:49:32 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-06-15 11:49:33 INFO: Loading these models for language: la (Latin):
| Processor | Package |
-----------------------
| tokenize  | ittb    |

2023-06-15 11:49:33 WARNING: GPU requested, but is not available!
2023-06-15 11:49:33 INFO: Using device: cpu
2023-06-15 11:49:33 INFO: Loading: tokenize
2023-06-15 11:49:33 INFO: Done loading processors!
